## Build Functions for ELT

In [1]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import modules as f # f is for function

%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [3]:
# manufactoring suppliers 
# 'lrcs', 'klac', ''

In [31]:
f.predictions('tsm')

TSM 5m Interval Timestamp: 2024-11-06 14:52:13 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,down,down,down
kelly_1:2.5,0.132,0.146555,0.159389,0.107357
prob_up,0.382903,0.330314,0.30541,0.125
prob_static,0.231165,0.274132,0.296269,0.25
prob_down,0.385932,0.395554,0.39832,0.625
precision,0.38,0.390397,0.399563,0.362398
recall,0.321429,0.351504,0.343985,0.25
f1,0.348269,0.369931,0.369697,0.295884
support,"[534.0, 532.0, 532.0]","[534.0, 532.0, 532.0]","[534.0, 532.0, 532.0]","[534.0, 532.0, 532.0]"


TSM 15m Interval Timestamp: 2024-11-06 14:52:21 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,static,static,static
kelly_1:2.5,0.198621,0.240359,0.2552,0.180786
prob_up,0.539745,0.348699,0.349604,0.375
prob_static,0.279852,0.443793,0.349863,0.375
prob_down,0.180403,0.207508,0.300533,0.25
precision,0.427586,0.457399,0.468,0.414847
recall,0.354286,0.573034,0.657303,0.533708
f1,0.3875,0.508728,0.546729,0.46683
support,"[175.0, 178.0, 176.0]","[175.0, 178.0, 176.0]","[175.0, 178.0, 176.0]","[175.0, 178.0, 176.0]"


TSM 30m Interval Timestamp: 2024-11-06 14:52:26 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,up
kelly_1:2.5,0.086316,0.062948,0.082062,0.081342
prob_up,0.344845,0.350929,0.399997,0.375
prob_static,0.330482,0.316791,0.296158,0.25
prob_down,0.324673,0.332279,0.303844,0.375
precision,0.347368,0.330677,0.34433,0.343816
recall,0.364238,0.366446,0.368653,0.362031
f1,0.355603,0.347644,0.356077,0.352688
support,"[453.0, 453.0, 450.0]","[453.0, 453.0, 450.0]","[453.0, 453.0, 450.0]","[453.0, 453.0, 450.0]"


TSM 1wk Interval Timestamp: 2024-11-06 14:52:55 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,up,up,static
kelly_1:2.5,0.071739,0.077273,0.121569,0.111304
prob_up,0.232061,0.390935,0.347745,0.25
prob_static,0.555062,0.307916,0.322798,0.625
prob_down,0.212878,0.301149,0.329457,0.125
precision,0.336957,0.340909,0.372549,0.365217
recall,0.333333,0.326087,0.413043,0.451613
f1,0.335135,0.333333,0.391753,0.403846
support,"[92.0, 93.0, 91.0]","[92.0, 93.0, 91.0]","[92.0, 93.0, 91.0]","[92.0, 93.0, 91.0]"


TSM 1mo Interval Timestamp: 2024-11-06 14:52:58 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,down,down,static
kelly_1:2.5,0.346667,0.030769,0.109091,0.187097
prob_up,0.478227,0.291664,0.195179,0.375
prob_static,0.147036,0.309957,0.360735,0.375
prob_down,0.374736,0.398379,0.444086,0.25
precision,0.533333,0.307692,0.363636,0.419355
recall,0.421053,0.210526,0.210526,0.619048
f1,0.470588,0.25,0.266667,0.5
support,"[19.0, 21.0, 19.0]","[19.0, 21.0, 19.0]","[19.0, 21.0, 19.0]","[19.0, 21.0, 19.0]"


###### Check Summary Output

In [ ]:
symbol='NVDA'
interval='1h'

f.transform(symbol, interval)

df = f.load_model_df(symbol,interval)

df.tail(7)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Select columns ending with 'z21'
z_columns = ['ac_z5', 'ac_z8', 'ac_z13', 'ac_z21', 'top_z21', 'body_z21', 'bottom_z21', 'vol_z21', 'pct_gap_up_down_z21', 'kma_sma40_diff_z21']
data_z21 = df[z_columns].dropna()  # Drop rows with missing values for simplicity

# Create an elbow plot to find the optimal number of clusters
inertia = []
cluster_range = range(1, 11)  # Test for clusters from 1 to 10

for k in cluster_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data_z21)
    inertia.append(kmeans.inertia_)

# Plot the elbow curve
plt.figure(figsize=(10, 6))
plt.plot(cluster_range, inertia, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Plot for Optimal Number of Clusters')
plt.xticks(cluster_range)
plt.grid(True)
plt.show()


In [ ]:
optimal_k = 3  # Replace with the optimal number from the elbow plot
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
data_z21['cluster'] = kmeans.fit_predict(data_z21)

# Check the cluster distribution
print(data_z21['cluster'].value_counts())

In [ ]:
# Add the 'cluster' column back to the original DataFrame
df['cluster'] = data_z21['cluster']



In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')